In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("data.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


**Split train and test set**

In [3]:
df.drop(df.columns[[-1, 0]], axis=1, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    object 
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

In [4]:
X = df[['radius_mean', 'perimeter_mean', 'area_mean', 'concavity_mean', 'concave points_mean']]
y = df[['diagnosis']]

In [5]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   diagnosis  569 non-null    object
dtypes: object(1)
memory usage: 4.6+ KB


In [6]:
yes_no_columns = ['diagnosis']
for col in yes_no_columns:
    y[col].replace({'B': 0,'M': 1},inplace=True)

C:\Users\gaura\Desktop\ANPR\anpr\lib\site-packages\pandas-1.3.1-py3.8-win-amd64.egg\pandas\core\generic.py:6610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [7]:
y.value_counts()

diagnosis
0            357
1            212
dtype: int64

In [8]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   diagnosis  569 non-null    int64
dtypes: int64(1)
memory usage: 4.6 KB


In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   radius_mean          569 non-null    float64
 1   perimeter_mean       569 non-null    float64
 2   area_mean            569 non-null    float64
 3   concavity_mean       569 non-null    float64
 4   concave points_mean  569 non-null    float64
dtypes: float64(5)
memory usage: 22.4 KB


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=25)

**Preprocessing: Scale the data so that both age and affordibility are in same scaling range**

In [11]:
X_train_scaled = X_train.copy()
X_train_scaled['radius_mean'] = X_train_scaled['radius_mean'] / 100  
X_train_scaled['perimeter_mean'] = X_train_scaled['perimeter_mean'] / 100
X_train_scaled['area_mean'] = X_train_scaled['area_mean'] / 100
X_train_scaled['concavity_mean'] = X_train_scaled['concavity_mean'] / 100
X_train_scaled['concave points_mean'] = X_train_scaled['concave points_mean'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['radius_mean'] = X_test_scaled['radius_mean'] / 100  
X_test_scaled['perimeter_mean'] = X_test_scaled['perimeter_mean'] / 100
X_test_scaled['area_mean'] = X_test_scaled['area_mean'] / 100
X_test_scaled['concavity_mean'] = X_test_scaled['concavity_mean'] / 100
X_test_scaled['concave points_mean'] = X_test_scaled['concave points_mean'] / 100

In [12]:
X_train_scaled.count()

radius_mean            455
perimeter_mean         455
area_mean              455
concavity_mean         455
concave points_mean    455
dtype: int64

In [13]:
y_train.count()

diagnosis    455
dtype: int64

In [14]:
model = keras.Sequential([
#     keras.layers.Dense(1, input_shape=(5,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros'),
    keras.layers.Dense(128, input_shape=(5,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=500)

Epoch 1/500
15/15 [==============================] - 1s 7ms/step - loss: 0.7196 - accuracy: 0.5780
Epoch 2/500
15/15 [==============================] - 0s 6ms/step - loss: 0.7023 - accuracy: 0.5846
Epoch 3/500
15/15 [==============================] - 0s 7ms/step - loss: 0.6821 - accuracy: 0.6132
Epoch 4/500
15/15 [==============================] - 0s 6ms/step - loss: 0.6912 - accuracy: 0.5473
Epoch 5/500
15/15 [==============================] - 0s 7ms/step - loss: 0.6935 - accuracy: 0.6154
Epoch 6/500
15/15 [==============================] - 0s 7ms/step - loss: 0.6650 - accuracy: 0.6044
Epoch 7/500
15/15 [==============================] - 0s 6ms/step - loss: 0.6638 - accuracy: 0.6242
Epoch 8/500
15/15 [==============================] - 0s 7ms/step - loss: 0.6891 - accuracy: 0.5604
Epoch 9/500
15/15 [==============================] - 0s 7ms/step - loss: 0.6748 - accuracy: 0.6154
Epoch 10/500
15/15 [==============================] - 0s 7ms/step - loss: 0.6672 - accuracy: 0.6220
Epoch 11/

15/15 [==============================] - 0s 7ms/step - loss: 0.4261 - accuracy: 0.8022
Epoch 84/500
15/15 [==============================] - 0s 7ms/step - loss: 0.4460 - accuracy: 0.7890
Epoch 85/500
15/15 [==============================] - 0s 7ms/step - loss: 0.4106 - accuracy: 0.8198
Epoch 86/500
15/15 [==============================] - 0s 6ms/step - loss: 0.4337 - accuracy: 0.8132
Epoch 87/500
15/15 [==============================] - 0s 6ms/step - loss: 0.4286 - accuracy: 0.8132
Epoch 88/500
15/15 [==============================] - 0s 6ms/step - loss: 0.4170 - accuracy: 0.8242
Epoch 89/500
15/15 [==============================] - 0s 6ms/step - loss: 0.4319 - accuracy: 0.8000
Epoch 90/500
15/15 [==============================] - 0s 6ms/step - loss: 0.4061 - accuracy: 0.8176
Epoch 91/500
15/15 [==============================] - 0s 6ms/step - loss: 0.3918 - accuracy: 0.8242
Epoch 92/500
15/15 [==============================] - 0s 6ms/step - loss: 0.4082 - accuracy: 0.8198
Epoch 93/500


15/15 [==============================] - 0s 7ms/step - loss: 0.3817 - accuracy: 0.8440
Epoch 165/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3576 - accuracy: 0.8527: 0s - loss: 0.3402 - accuracy: 0.86
Epoch 166/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3655 - accuracy: 0.8527
Epoch 167/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3789 - accuracy: 0.8527
Epoch 168/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3522 - accuracy: 0.8571
Epoch 169/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3737 - accuracy: 0.8374
Epoch 170/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3867 - accuracy: 0.8352
Epoch 171/500
15/15 [==============================] - 0s 10ms/step - loss: 0.3598 - accuracy: 0.8615
Epoch 172/500
15/15 [==============================] - 0s 10ms/step - loss: 0.3772 - accuracy: 0.8462
Epoch 173/500
15/15 [==============================] - 0s 7ms/step 

15/15 [==============================] - 0s 7ms/step - loss: 0.3531 - accuracy: 0.8440
Epoch 245/500
15/15 [==============================] - 0s 6ms/step - loss: 0.3877 - accuracy: 0.8308
Epoch 246/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3438 - accuracy: 0.8659
Epoch 247/500
15/15 [==============================] - 0s 6ms/step - loss: 0.3443 - accuracy: 0.8615
Epoch 248/500
15/15 [==============================] - 0s 6ms/step - loss: 0.3605 - accuracy: 0.8330
Epoch 249/500
15/15 [==============================] - 0s 6ms/step - loss: 0.3653 - accuracy: 0.8286
Epoch 250/500
15/15 [==============================] - 0s 6ms/step - loss: 0.3467 - accuracy: 0.8418
Epoch 251/500
15/15 [==============================] - 0s 6ms/step - loss: 0.3833 - accuracy: 0.8286
Epoch 252/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3349 - accuracy: 0.8659
Epoch 253/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3614 - accuracy: 0.8374
Epoc

15/15 [==============================] - 0s 9ms/step - loss: 0.3209 - accuracy: 0.8615
Epoch 326/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3285 - accuracy: 0.8637
Epoch 327/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3501 - accuracy: 0.8462
Epoch 328/500
15/15 [==============================] - 0s 6ms/step - loss: 0.3252 - accuracy: 0.8615
Epoch 329/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3375 - accuracy: 0.8505
Epoch 330/500
15/15 [==============================] - 0s 6ms/step - loss: 0.3389 - accuracy: 0.8505
Epoch 331/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3477 - accuracy: 0.8615
Epoch 332/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3269 - accuracy: 0.8703
Epoch 333/500
15/15 [==============================] - 0s 6ms/step - loss: 0.3720 - accuracy: 0.8242
Epoch 334/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3390 - accuracy: 0.8484
Epoc

15/15 [==============================] - 0s 7ms/step - loss: 0.3266 - accuracy: 0.8527
Epoch 407/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3306 - accuracy: 0.8681
Epoch 408/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3511 - accuracy: 0.8462
Epoch 409/500
15/15 [==============================] - 0s 6ms/step - loss: 0.3336 - accuracy: 0.8681
Epoch 410/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3334 - accuracy: 0.8571
Epoch 411/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3114 - accuracy: 0.8681
Epoch 412/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3207 - accuracy: 0.8703
Epoch 413/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3345 - accuracy: 0.8549
Epoch 414/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3472 - accuracy: 0.8462
Epoch 415/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3267 - accuracy: 0.8637
Epoc

15/15 [==============================] - 0s 8ms/step - loss: 0.3179 - accuracy: 0.8637
Epoch 488/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3128 - accuracy: 0.8659
Epoch 489/500
15/15 [==============================] - 0s 6ms/step - loss: 0.3272 - accuracy: 0.8527
Epoch 490/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3445 - accuracy: 0.8440
Epoch 491/500
15/15 [==============================] - 0s 6ms/step - loss: 0.3400 - accuracy: 0.8549
Epoch 492/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3382 - accuracy: 0.8703
Epoch 493/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3191 - accuracy: 0.8681
Epoch 494/500
15/15 [==============================] - 0s 6ms/step - loss: 0.3152 - accuracy: 0.8571
Epoch 495/500
15/15 [==============================] - 0s 6ms/step - loss: 0.3490 - accuracy: 0.8549
Epoch 496/500
15/15 [==============================] - 0s 6ms/step - loss: 0.3529 - accuracy: 0.8527
Epoc

**Evaluation of the model on test set**

In [15]:
X_test_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114 entries, 522 to 360
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   radius_mean          114 non-null    float64
 1   perimeter_mean       114 non-null    float64
 2   area_mean            114 non-null    float64
 3   concavity_mean       114 non-null    float64
 4   concave points_mean  114 non-null    float64
dtypes: float64(5)
memory usage: 5.3 KB


In [16]:
X_test_scaled.count()

radius_mean            114
perimeter_mean         114
area_mean              114
concavity_mean         114
concave points_mean    114
dtype: int64

In [17]:
y_test.count()

diagnosis    114
dtype: int64

In [18]:
model.evaluate(X_test_scaled,y_test)

4/4 [==============================] - 0s 5ms/step - loss: 0.2908 - accuracy: 0.9035


[0.29077550768852234, 0.9035087823867798]

In [19]:
model.predict(X_test_scaled)

array([[0.05922468],
       [0.06722688],
       [0.12599166],
       [0.1486426 ],
       [0.259195  ],
       [0.05217424],
       [0.01793312],
       [0.03220113],
       [0.05455922],
       [0.1457286 ],
       [0.09512722],
       [0.03232838],
       [0.42883334],
       [0.99772567],
       [0.9481014 ],
       [0.22261198],
       [0.9773497 ],
       [0.1136056 ],
       [0.0807891 ],
       [0.99060166],
       [0.2637024 ],
       [0.44462198],
       [0.03570955],
       [0.99904007],
       [0.99939644],
       [0.99726415],
       [0.02450713],
       [0.20183982],
       [0.12933919],
       [0.99914396],
       [0.25640061],
       [0.02784302],
       [0.02830883],
       [0.64922893],
       [0.9863868 ],
       [0.99866474],
       [0.06550326],
       [0.02085248],
       [0.9802917 ],
       [0.43666497],
       [0.09283278],
       [0.47937807],
       [0.36499345],
       [0.4245177 ],
       [0.36411107],
       [0.1766988 ],
       [0.6495719 ],
       [0.463

In [22]:
model.save('model')

INFO:tensorflow:Assets written to: model\assets


In [24]:
import tensorflow as tf
from keras.models import load_model

model.save('model.h5')

**Value of weights and bias from the model**

In [ ]:
coef, intercept = model.get_weights()

In [ ]:
coef, intercept